# Meeting minutes creator

In this colab, we make a meeting minutes program.

It includes useful code to connect your Google Drive to your colab.

Upload your own audio to make this work!!

https://colab.research.google.com/drive/1KSMxOCprsl1QRpt_Rq0UqCAyMtPqDQYx?usp=sharing

This should run nicely on a low-cost or free T4 box.

In [2]:
!pip install -q gradio transformers torch librosa

In [8]:
from dotenv import load_dotenv
import gradio as gr
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import librosa
from transformers import pipeline
from openai import OpenAI
import os

# Load the Whisper model for audio transcription
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# Set up OpenAI API key (replace with your actual key)
load_dotenv()
 # Store your API key in an environment variable
openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def transcribe_audio(audio_file):
    # Load the audio file
    audio, sr = librosa.load(audio_file, sr=16000)
    
    # Process the audio
    input_features = processor(audio, sampling_rate=sr, return_tensors="pt").input_features
    
    # Generate transcription
    predicted_ids = model.generate(input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    
    return transcription

def generate_summary(transcription):
    # Define system and user prompts
    system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways, and action items with owners, in markdown."
    user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location, and date; discussion points; takeaways; and action items with owners.\n{transcription}"
    
    # Call OpenAI API
    response = openai.chat.completions.create(
        model="gpt-4o-mini",  # Use GPT-3.5-turbo for summarization
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=1000,  # Adjust as needed
        temperature=0.5   # Adjust for creativity vs. determinism
    )
    
    # Extract the generated summary
    summary = response.choices[0].message.content
    return summary

def process_audio(file):
    # Step 1: Transcribe the audio
    transcription = transcribe_audio(file)
    
    # Step 2: Generate a summary
    summary = generate_summary(transcription)
    
    return summary

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# Audio Transcription and Summarization")
    
    with gr.Row():
        audio_input = gr.Audio(type="filepath", label="Upload MP3 File")
        markdown_output = gr.Markdown(label="Summary")
    
    submit_button = gr.Button("Submit")
    
    submit_button.click(fn=process_audio, inputs=audio_input, outputs=markdown_output)

# Launch the Gradio app
demo.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
